> [Deep](https://youtu.be/GYecDQQwTdI?list=PLQ28Nx3M4Jrguyuwg4xe9d9t2XE639e5C)

# Xor prob

코드가 조금 더러워요..

In [63]:
import numpy as np
import tensorflow as tf

w1 = np.array([5, 5])
b1 = -8
w2 = np.array([-7, -7])
b2 = 3
w3 = np.array([-11, -11])
b3 = 6

x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
a, b = np.matmul(x, w1) + b1, np.matmul(x, w2) + b2
c = np.array([a, b])
c = c.swapaxes(0, 1)
print(c)
c = tf.sigmoid(c.astype(np.float32))
print(c)

result = np.matmul(c, w3) + b3

result = tf.sigmoid(result)

print(tf.cast(result >= 0.5, tf.float32))

[[ -8   3]
 [ -3  -4]
 [ -3  -4]
 [  2 -11]]
tf.Tensor(
[[3.3542514e-04 9.5257413e-01]
 [4.7425866e-02 1.7986238e-02]
 [4.7425866e-02 1.7986238e-02]
 [8.8079709e-01 1.6701701e-05]], shape=(4, 2), dtype=float32)
tf.Tensor([0. 1. 1. 0.], shape=(4,), dtype=float32)


무튼 대수적으로 가능

이런 W와 b는 많을 것이다아

코드를 매트릭스 써서 좀더 깔끔하게

In [64]:
import tensorflow as tf
import numpy as np

w1 = np.array([
  [5, -7],
  [5, -7]
], dtype=np.float32)
b1 = np.array([-8, 3], dtype=np.float32)

w2 = np.array([[-11], [-11]], dtype=np.float32)
b2 = 6.

x = np.array([
  [0, 0],
  [0, 1],
  [1, 0],
  [1, 1],
], dtype=np.float32)

l1 = tf.sigmoid(tf.matmul(x, w1) + b1)
l2 = tf.sigmoid(tf.matmul(l1, w2) + b2)

print(tf.cast(l2 > 0.5, tf.int32).numpy())

[[0]
 [1]
 [1]
 [0]]


매우 깔끔!

그럼 이걸 어떻게 학습 시킬까..?

-> Gradient Descent

합성함수를 미분하는 것과 같다.

cost함수를 weight에 따라 미분하면 된다.

미분 지옥

# LAB

## training

In [65]:
import numpy as np
import tensorflow as tf

x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]

In [66]:
dataset = tf.data.Dataset.from_tensor_slices((x_data, y_data)).batch(len(x_data))

def preprocess_data(features, labels):
  features = tf.cast(features, tf.float32)
  labels = tf.cast(labels, tf.float32)
  return features, labels

In [67]:
W1 = tf.Variable(tf.random.normal([2, 1]))
b1 = tf.Variable(tf.random.normal([1]))

W2 = tf.Variable(tf.random.normal([2, 1]))
b2 = tf.Variable(tf.random.normal([1]))

W3 = tf.Variable(tf.random.normal([2, 1]))
b3 = tf.Variable(tf.random.normal([1]))

In [74]:
def neural_net(features):
  layer1 = tf.sigmoid(tf.matmul(features, W1) + b1)
  layer2 = tf.sigmoid(tf.matmul(features, W2) + b2)
  layer3 = tf.concat([layer1, layer2], -1)
  layer3 = tf.reshape(layer3, [-1, 2])

  hypothesis = tf.sigmoid(tf.matmul(layer3, W3) + b3)
  return hypothesis

def loss_fn(hypothesis, labels):
  cost = -(tf.reduce_mean(labels * tf.math.log(hypothesis)
    + (1 - labels) * tf.math.log(1 - hypothesis)))
  return cost

optimizer = tf.optimizers.SGD()

def accuracy_fn(hypothesis, labels):
  predicted = tf.cast(hypothesis >= 0.5, tf.float32)
  accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, hypothesis), tf.float32))
  return accuracy

def grad(hypothesis, features, labels):
  with tf.GradientTape() as tape:
    loss_value = loss_fn(neural_net(features), labels)
  return tape.gradient(loss_value, [W1, W2, W3, b1, b2, b3]), loss_value

In [75]:
EPOCHS = 50000

for step in range(EPOCHS + 1):
  for features, labels in dataset:
    features, labels = preprocess_data(features, labels)
    grads, loss = grad(None, features, labels)
    optimizer.apply_gradients(zip(grads, [W1, W2, W3, b1, b2, b3]))
    if step % 5000 == 0:
      print("Iter : {:5}, Loss {:.4f}".format(step, loss))

Iter :     0, Loss 0.7253
Iter :  5000, Loss 0.6856
Iter : 10000, Loss 0.6698
Iter : 15000, Loss 0.6329
Iter : 20000, Loss 0.5658
Iter : 25000, Loss 0.4867
Iter : 30000, Loss 0.3522
Iter : 35000, Loss 0.2031
Iter : 40000, Loss 0.1218
Iter : 45000, Loss 0.0823
Iter : 50000, Loss 0.0609


## Tensorboard

각종 변수 값, graph flow등을 확인 할 수 있다.